# An Explain Report

This is a POC for an explain report: given a number at time T1 and time T2, introspect the clocks involved to produce a simple linear breakdown
of what changed on various timelines to account for the change in the number between T1 and T2.

The report is an ordered sequence of clock changes: the sum of changes should equal to the total change. The order of application is important: change the
ordering, and the resultant values may change. It's not a Jacobian-type report with T1 deltas and no ordering.

The real reason for this workbook is to start working on the node class optimizations and metadata management.

## Footnotes

Footnotes are one of the simplest types of metadata:
* Any computation (even a simple read of a constant value) may declare it has footnotes
* Every computation gets the footnote of all its inputs

Footnoting is based on a number of observations about large software systems (say, 1M LOC or more):
* Any complex report (i.e. computation output) will generally be somewhat wrong, misleading, or out of date.
* If you want 100% correctness and truth, your report will raise an exception every time you run it.
* If you just want it to run, you'll get a result, but some log file on some machine in some compute farm will have a message explaining why your result is wrong.
* And you won't see that message.
* And that message may have leaked information that bad actors can read.
* And you won't trust this system, so your own developers will copy the underlying data and write their own version of the report with all the same issues.
* Now you don't have a problem, but your company now has two problems.

Footnotes summary:
* Are the documentation of problems from the producer's point of view
* Make no claim about usability of results, etc, for a specific consumer
* May be programmatically removed/condensed/replaced at controlled code points


In [1]:
from mand.core import Entity, node, Context
from mand.core import ObjectDb, Timestamp, getNode, _tr
from mand.core import ProfileMonitor
from mand.core import displayDict, displayMarkdown, displayListOfDicts, displayHeader
from mand.core import find
from mand.demos.trading import makeWorld, bookSomeTrades

db = ObjectDb()

from mand.lib.dbsetup import setUpDb
setUpDb(db)
db.describe()

<mand.db.ObjectDb object at 0x105d3e510>: 23, mem=True, ro=False: entities=9, map=2


In [2]:
with db:
    pWorld = makeWorld()
    
pAll, bExt, bExt2 = pWorld.children()
p1 = pAll.children()[0]
p2 = pAll.children()[1]
p4 = pAll.children()[3]

b2 = p2.children()[0]
b4 = p4.children()[0]

makeWorld, TopOfTheHouse is: <Entity:/Global/TradingPortfolio/TopOfTheHouse>
    # books: 100
    # children: 10


# Book some trades

This time, we throw a few thousand in...

In [3]:
ts0, ts1, ts2, ts3, ts4, ts5, eod, ts6 = bookSomeTrades(pWorld)

# An Explain Report

This is a mix of abstractions at the Core, DBA, and BA, and User levels. But, it does the job for now...

In [4]:
class Report(Entity):
    @node(stored=True)
    def valuable(self):
        return None
    
    @node(stored=True)
    def ts1(self):
        return None
    
    @node(stored=True)
    def ts2(self):
        return None
    
    @node
    def cutoffs(self):
        valuable = self.valuable()
        ts1 = self.ts1()
        ts2 = self.ts2()
        
        clock = valuable.getObj(_tr.RootClock, 'Main')
        
        def cuts(ts):
            def fn(node):
                obj = node.key.object()
                m = node.key.shortName()
                if isinstance(obj, _tr.Clock) and m == 'cutoffs':
                    return True
            with Context({clock.cutoffs: ts}, 'Clocks'):
                nodes = find(valuable.NPV, fn)
                return nodes

        ret = {}
        for n in cuts(ts1):
            ret[n.key._key] = n
        for n in cuts(ts2):
            ret[n.key._key] = n
        return ret.values()
    
    @node
    def data(self):
        valuable = self.valuable()
        ts1 = self.ts1()
        ts2 = self.ts2()
        clock = valuable.getObj(_tr.RootClock, 'Main')
    
        nodes = self.cutoffs()
    
        # IRL, we'd sort these according to some business req...
        # And our clocks might be arranged in an N-level tree...
        nodes = sorted(nodes, key = lambda node: node.object().meta.name())
    
        data = []
        curr = [0]
        def add(title, npv):
            pnl = npv - curr[0]
            curr[0] = npv
            data.append( {'Activity': title, 'PnL': pnl } )

        with Context({clock.cutoffs: ts1}, 'Start'):
            curr = [ valuable.NPV() ] # Starting balance
    
        tweaks = {}
        for n in nodes:
            tweaks[n] = ts1
        with Context(tweaks, name='Start breaks'):
            start = valuable.NPV()
            add('Starting balance breaks', start)

        tsAmend = Timestamp(t=ts2.transactionTime, v=ts1.validTime)
        # XXX - modifying tweaks in place is a bit evil
        # This is only safe because I know Context() effectively copies, so this works
        # for now.
        for n in nodes:
            tweaks[n] = tsAmend
            name = n.object().meta.name()
            with Context(tweaks, name='Amend %s' % name):
                add('prior day amends: %s' % name, valuable.NPV())
        for n in nodes:
            tweaks[n] = ts2
            name = n.object().meta.name()
            with Context(tweaks, name='Activity %s' % name):
                add('activity: %s' % name, valuable.NPV())
    
        with Context({clock.cutoffs: ts2}, name='End'):
            end = valuable.NPV()
            add('Ending balance breaks', end)

        title = 'PnL explain for %s: %s' % (valuable.meta.name(), end-start)
        return data, title

    def run(self):
        data, title = self.data()
        node = getNode(self.data)
        footnotes = node.footnotes.values()
        displayHeader('%s' % title)
        if footnotes:
            displayMarkdown('**Caveat: this report encountered problems. See footnotes at bottom.**')
        displayListOfDicts(data, names=['Activity', 'PnL'] )
        if footnotes:
            displayMarkdown('## Footnotes:')
            displayMarkdown(node.footnoteMarkdown())
    
r = Report(valuable=pAll, ts1=eod, ts2=ts6)
r.run()

# PnL explain for TopOfTheHouse: 5236.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|0.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Footnotes:

1. Inadequate cash discounting model used

## Add some inconsistent data [Test]

Book b2 should appear multiple times in some portfolio trees and be flagged accordingly...

In [5]:
with db:
    p1.setChildren(p1.children() + [b2])

ts7 = Timestamp()

# Footnotes

Note the report calculation has run, but attached appropriate caveats to the output:

In [6]:
db3 = db.copy()
p = db3._get(pAll.meta.path())

with ProfileMonitor(mode='sum'): 
    r = Report(valuable=p, ts1=eod, ts2=ts7)
    r.run()

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Footnotes:

1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|Report:data|1|20,064,079|17|20,064,079|GetValue
|Report:data|1|20,064,062|423|20,064,062|GetValue/Calc
|TradingContainer:NPV|11|19,999,269|134|1,818,115|GetValue
|TradingContainer:NPV|11|19,999,135|81|1,818,103|GetValue/Calc
|Portfolio:items|121|19,825,515|1,100|163,847|GetValue
|Portfolio:items|121|19,825,425|784|163,846|GetValue/Calc
|Workbook:items|1,104|18,797,960|10,949|17,027|GetValue
|Workbook:items|1,100|18,787,011|7,786|17,079|GetValue/Calc
|Report:cutoffs|1|11,397,364|19|11,397,364|GetValue
|Report:cutoffs|1|11,397,345|138|11,397,345|GetValue/Calc
|Root:Clocks|548|11,397,192|1,839,108|20,797|Context
|TradeOpenEvent:ticket|22,286|4,510,472|110,753|202|GetValue
|TradeOpenEvent|1,014|4,428,549|4,428,549|4,367|Db.Get
|TradingTicket|1,014|4,399,719|4,399,719|4,338|Db.Get
|Root:Activity MarketData|255|1,286,013|1,218,617|5,043|Context
|Root:Start breaks|255|1,199,103|1,129,683|4,702|Context
|Portfolio:children|242|987,752|1,494|4,081|GetValue
|Portfolio:children|121|986,258|931|8,150|GetValue/Calc
|Root:Activity Trading|263|920,603|845,114|3,500|Context
|Root:End|278|894,429|825,813|3,217|Context
|Root:Start|270|891,669|824,703|3,302|Context
|Root:Amend MarketData|255|873,615|806,117|3,425|Context
|Root:Amend Portfolio|255|869,740|803,157|3,410|Context
|Root:Activity Portfolio|263|864,579|797,646|3,287|Context
|Root:Amend Trading|255|861,473|794,311|3,378|Context
|PortfolioUpdateEvent:children|121|410,147|2,831|3,389|GetValue
|TradingBook|102|377,114|377,114|3,697|Db.Get
|Equity:NPV|15|171,396|162|11,426|GetValue
|Equity:NPV|15|171,233|101|11,415|GetValue/Calc
|MarketInterface:spot|15|162,660|149|10,844|GetValue
|MarketInterface:spot|15|162,510|95|10,834|GetValue/Calc
|ExternalRefData:state|15|152,832|125|10,188|GetValue
|ExternalRefData:state|15|152,706|100|10,180|GetValue/Calc
|RefData:state|15|151,825|134|10,121|GetValue
|RefData:state|15|151,691|101|10,112|GetValue/Calc
|_WorkItemEvent:book2|22,286|107,613|101,067|4|GetValue
|Clock:cutoffs|2,480|103,119|8,506|41|GetValue
|_WorkItemEvent:book1|22,286|99,025|99,025|4|GetValue
|Clock:cutoffs|20|94,683|126|4,734|GetValue/Calc
|Clock:parent|20|93,746|185|4,687|GetValue
|Clock:parent|20|93,560|147|4,678|GetValue/Calc
|TradeOpenEvent:quantity|22,286|86,948|86,948|3|GetValue
|_WorkItemEvent:item|11,143|75,723|68,906|6|GetValue
|TradeOpenEvent:action|11,143|68,691|68,691|6|GetValue
|TradeOpenEvent:premium|11,143|67,142|62,621|6|GetValue
|PortfolioUpdateEvent|12|57,295|57,295|4,774|Db.Get
|TradingBook:clock|2,200|56,292|20,102|25|GetValue
|Event:amends|11,340|55,978|55,978|4|GetValue
|TradeOpenEvent:unitPrice|11,143|55,681|55,681|4|GetValue
|RefDataUpdateEvent|9|37,975|37,975|4,219|Db.Get
|TradingPortfolio|10|36,746|36,746|3,674|Db.Get
|TradingBook:clock|1,100|36,189|10,735|32|GetValue/Calc
|Clock|5|17,665|17,665|3,533|Db.Get
|Portfolio:clock|242|9,122|2,129|37|GetValue
|MarketInterface:source|15|9,020|117|601|GetValue
|MarketInterface:source|15|8,902|94|593|GetValue/Calc
|ClockEvent|2|8,147|8,147|4,073|Db.Get
|MarketDataSource|2|8,081|8,081|4,040|Db.Get
|Equity:refdata|15|7,829|165|521|GetValue
|Portfolio:books|231|7,825|1,512|33|GetValue
|Equity:refdata|15|7,663|100|510|GetValue/Calc
|Portfolio:clock|121|6,993|1,121|57|GetValue/Calc
|MarketInterface|2|6,929|6,929|3,464|Db.Get
|ClockEvent:parent|8|6,818|75|852|GetValue
|Equity|2|6,817|6,817|3,408|Db.Get
|Portfolio:books|121|6,632|773|54|GetValue/Calc
|RootClock|1|5,031|5,031|5,031|Db.Get
|ForwardCashflow|1|4,521|4,521|4,521|Db.Get
|MarketDataSource:clock|30|4,493|258|149|GetValue
|MarketDataSource:clock|15|4,235|155|282|GetValue/Calc
|Entity:clock|40|929|377|23|GetValue
|Entity:clock|20|551|202|27|GetValue/Calc
|ForwardCashflow:NPV|11|310|120|28|GetValue
|MarketInterface:sourceName|15|299|125|19|GetValue
|RefDataUpdateEvent:data|58|245|245|4|GetValue
|ForwardCashflow:NPV|11|190|73|17|GetValue/Calc
|RootClock:cutoffs|52|180|180|3|GetValue
|MarketInterface:sourceName|15|174|104|11|GetValue/Calc
|Equity:assetName|15|106|106|7|GetValue
|Report:valuable|2|12|12|6|GetValue
|Report:ts2|2|7|7|3|GetValue
|Report:ts1|2|7|7|3|GetValue